In [13]:
import re
import time
import pandas as pd
import requests

from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains



def melon_crawling(song_num, page_iter, genre, csv_name, myID, pwd):
    
    # Mozilla
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}

    # 시작할 주소와 곡 넘버
    song_num = song_num
    url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex='
    # 저장할 DataFrame생성 
    song_df = pd.DataFrame([], columns=['곡명', '가수', '장르', '태그', '발매일', '좋아요', '가사'])
    
    
    # 50곡마다 1페이지를 이용하여 페이지를 넘김
    for page_num in range(page_iter):
        # 크롬 웹드라이브 설정
        # driver1은 노래 목록 사이트
        # driver2는 각각의 노래 페이지를 열어줌
        driver1 = webdriver.Chrome('chromedriver.exe')
        driver2 = webdriver.Chrome('chromedriver.exe')   # chrome_options=options
        driver1.get(url + str(song_num + 50 * page_num))

        element = WebDriverWait(driver1, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > div > div > a.last.on")))
        
        # 인기순의 클릭이 되지 않으면 넘어가지 않도록
#         driver1.find_elements_by_xpath('//*[@id="frm"]/div/div/div/a[2]/span')
#         print(driver1.current_url)
#         print(driver1.page_source)
        
        # 멜론 주소를 BeautifulSoup로 처리
        melon = bs(driver1.page_source, 'lxml')
        lst_song = melon.select('div[id="songList"] > form[id="frm"] > div > table > tbody > tr')
        
        for song in lst_song:
            id_song = song.select_one('a[class="btn button_icons type03 song_info"]')['href'].replace('javascript:melon.link.goSongDetail', '')
            id_number = re.sub("[(\)\'\;]", '', id_song)
            
            # 가져온 url로 bs에 할당
            url_song = 'https://www.melon.com/song/detail.htm?songId=' + id_number
            driver2.get(url_song)
            driver2.implicitly_wait(10)
            song = driver2.page_source
            soup_song = bs(song, 'lxml')


            # 노래 제목
            # 19금 노래라면
            if soup_song.find('span', attrs={'class':'none'}):
                song_title = re.sub('[곡명	, 19금]', '', soup_song.find('div', attrs={'class':'song_name'}).text).strip()
            # 19금 노래가 아니라면
            else:
                song_title = re.sub('[곡명	]', '', soup_song.find('div', attrs={'class':'song_name'}).text).strip()
            # print(song_title)


            # 가수
            singer_list = []
            singers = soup_song.select('div[class="artist"] > a > span')
            for n, singer in enumerate(singers):
                if n % 2 == 0:
                    singer_list.append(singer.text)
            # print(singer_list)


            # 수집하는 장르
            genre = genre


            # 장르(하위 태그)
            tag = soup_song.select('dl[class="list"] > dd')[2].text


            # 발매일
            release_date = soup_song.select('dl[class="list"] > dd')[1].text
            # print(release_date)


            # 좋아요
            like_count = re.sub('[,]', '', soup_song.find('div', attrs={'class':'button d_song_like'}).find('span', attrs={'id':'d_like_count'}).text)
            # print(like_count)


            # 가사
            # 성인 인증에서 막히면 멜론아이디로 로그인
            if soup_song.select('button[title="성인 인증"]'):
                driver2.find_element(By.XPATH, '//*[@id="lyricArea"]/div/button').click()
                driver2.find_element(By.XPATH, '//*[@id="conts_section"]/div/div/div[3]/button').click()
                driver2.find_element(By.XPATH, '//*[@id="id"]').send_keys(myID)
                driver2.find_element(By.XPATH, '//*[@id="pwd"]').send_keys(pwd)
                driver2.find_element(By.XPATH, '//*[@id="btnLogin"]').click()
                soup_song = bs(driver2.page_source, 'lxml')

            
            lyric = str(soup_song.select('div[class="lyric"]'))            
            
            # 가사가 없는건 제외
            if lyric == []:
                continue
            lyric = lyric.replace('<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->', '').\
                          replace('<div class="lyric" id="d_video_summary">', '').\
                          replace('	', '').\
                          replace('<br/>', '\n').\
                          replace('</div>', '')
            # print(lyric)

             # 데이터프레임 합치기
            song_detail = pd.DataFrame([[song_title, singer_list, genre, tag, release_date, like_count, lyric]], columns=['곡명', '가수', '장르', '태그', '발매일', '좋아요', '가사'])
            song_df = pd.concat((song_df, song_detail))
            
        print('현재까지 처리된 마지막 넘버:', (page_num + 1) * 50 + (song_num - 1))
    

    # csv파일로 DataFrame저장
    song_df.to_csv(csv_name, mode='w', encoding='utf-8-sig', index=False)
    
    # 중복된 가수와 곡명이 있는 데이터프레임 제외
    df = pd.read_csv(csv_name)
    if (df[['곡명', '가수']].duplicated() == True) is True:
        df.sort_values(by=['좋아요'], ascending=False, inplace=True).drop_duplicates(['곡명', '가수'], keep='first', inplace=True).sort_index()
        df.to_save(csv_name, mode='w', encoding='utf-8-sig', index=False)
    print('마지막 곡의 넘버:', page_iter * 50 + (song_num - 1))
    
    
melon_crawling(song_num=2501, page_iter=50, genre='랩_힙합', csv_name='랩_힙합(70001_75000).csv', myID='chosewan', pwd='dojjang0112')
    

<ipython-input-13-7a4b9abf93cd>:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome('chromedriver.exe')
<ipython-input-13-7a4b9abf93cd>:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome('chromedriver.exe')   # chrome_options=options


현재까지 처리된 마지막 넘버: 70050
현재까지 처리된 마지막 넘버: 70100
현재까지 처리된 마지막 넘버: 70150
현재까지 처리된 마지막 넘버: 70200
현재까지 처리된 마지막 넘버: 70250
현재까지 처리된 마지막 넘버: 70300
현재까지 처리된 마지막 넘버: 70350
현재까지 처리된 마지막 넘버: 70400
현재까지 처리된 마지막 넘버: 70450
현재까지 처리된 마지막 넘버: 70500
현재까지 처리된 마지막 넘버: 70550
현재까지 처리된 마지막 넘버: 70600
현재까지 처리된 마지막 넘버: 70650
현재까지 처리된 마지막 넘버: 70700
현재까지 처리된 마지막 넘버: 70750
현재까지 처리된 마지막 넘버: 70800
현재까지 처리된 마지막 넘버: 70850
현재까지 처리된 마지막 넘버: 70900
현재까지 처리된 마지막 넘버: 70950
현재까지 처리된 마지막 넘버: 71000
현재까지 처리된 마지막 넘버: 71050
현재까지 처리된 마지막 넘버: 71100
현재까지 처리된 마지막 넘버: 71150
현재까지 처리된 마지막 넘버: 71200
현재까지 처리된 마지막 넘버: 71250
현재까지 처리된 마지막 넘버: 71300
현재까지 처리된 마지막 넘버: 71350
현재까지 처리된 마지막 넘버: 71400
현재까지 처리된 마지막 넘버: 71450
현재까지 처리된 마지막 넘버: 71500
현재까지 처리된 마지막 넘버: 71550
현재까지 처리된 마지막 넘버: 71600
현재까지 처리된 마지막 넘버: 71650
현재까지 처리된 마지막 넘버: 71700
현재까지 처리된 마지막 넘버: 71750
현재까지 처리된 마지막 넘버: 71800
현재까지 처리된 마지막 넘버: 71850
현재까지 처리된 마지막 넘버: 71900
현재까지 처리된 마지막 넘버: 71950
현재까지 처리된 마지막 넘버: 72000
현재까지 처리된 마지막 넘버: 72050
현재까지 처리된 마지막 넘버: 72100
현재까지 처리된 마지막 넘버: 72150
현재까지 처리된 마지